In [1]:
%matplotlib qt
import os
import numpy as np
import matplotlib.pyplot as plt
import mne
from mne.time_frequency import tfr_morlet, psd_multitaper, psd_welch
from mne_bids import make_bids_basename

In [2]:
mne.set_log_level('WARNING')

In [3]:
sSub = '08'
session, task, run = '01', 'picturenaming', '01'
bids_basename = make_bids_basename(subject=sSub, session=session,
                                  task=task, run=run)
bids_fname = bids_basename + '_meg.fif'

data_dir = '/Users/chholak/data/pic-name-data-bids/'
res_dir = '/Users/chholak/research/results/pic_name/'
data_path = data_dir + 'MEG/'

bids_path = data_path + 'sub-' + sSub + '/ses-' + session + '/meg/'
epochs_fname = bids_path + bids_fname.replace('_meg.fif', '-epo.fif')

In [8]:
mne.read_epochs?

In [9]:
epochs = mne.read_epochs(epochs_fname, proj=True)
epochs

<EpochsFIF  |   98 events (all good), -0.5 - 1 sec, baseline [-0.5, 0], ~351.3 MB, data loaded,
 'picture': 98>

In [10]:
epochs.apply_proj()

<EpochsFIF  |   98 events (all good), -0.5 - 1 sec, baseline [-0.5, 0], ~351.3 MB, data loaded,
 'picture': 98>

In [6]:
epochs.plot_psd(fmin=2., fmax=40., average=False, bandwidth=2);

In [11]:
epochs.plot_psd_topomap(ch_type='mag', normalize=True, cmap='viridis');
# plt.savefig(res_dir + 'psd_topomap_mag_' + 'sub-' + sSub + '.pdf')

In [ ]:
epochs.plot_psd_topomap(ch_type='grad', normalize=True, cmap='viridis');
# plt.savefig(res_dir + 'psd_topomap_grad_' + 'sub-' + sSub + '.pdf')

In [ ]:
psds, freqs = psd_multitaper(epochs, fmin=2, fmax=40, n_jobs=1, bandwidth=2)
psds.shape

In [ ]:
psds = np.mean(10. * np.log10(psds), axis=0)
picks = mne.pick_types(epochs.info, meg='grad', eeg=False)

f, ax = plt.subplots()
ax.plot(freqs, psds[picks].T, color='k', alpha=0.3)
ax.set(title='Multitaper PSD', xlabel='Frequency (Hz)',
      ylabel='Power Spectral Density (dB)')
plt.show()

In [ ]:
freqs = np.logspace(*np.log10([2, 30]), num=20)
n_cycles = freqs / 2.
power, itc = tfr_morlet(epochs, freqs=freqs, n_cycles=n_cycles, use_fft=True,
                       return_itc=True, decim=3, n_jobs=1)

In [ ]:
power.crop(-0.1, 0.8)

In [ ]:
itc.crop(-0.1, 0.8)

In [ ]:
baseline_mode = 'logratio'
baseline = (None, 0)
power.plot_topo(baseline=baseline, mode=baseline_mode, title='Average power');

In [ ]:
for inx, name in enumerate(epochs.ch_names):
    if name == 'MEG2343':
        print("MEG 2343 is channel #", inx)

In [ ]:
power.plot([267], baseline=baseline, mode=baseline_mode);
plt.savefig(res_dir + 'power_MEG2343_' + 'sub-' + sSub + '.pdf')

In [ ]:
fig, axis = plt.subplots(1, 3, figsize=(7, 4))
power.plot_topomap(ch_type='grad', tmin=0., tmax=0.8, fmin=4, fmax=7,
                  baseline=baseline, mode=baseline_mode, axes=axis[0],
                  title='Theta', show=False, contours=1)
power.plot_topomap(ch_type='grad', tmin=0., tmax=0.8, fmin=8, fmax=12,
                  baseline=baseline, mode=baseline_mode, axes=axis[1],
                  title='Alpha', show=False, contours=1)
power.plot_topomap(ch_type='grad', tmin=0., tmax=0.8, fmin=15, fmax=30,
                  baseline=baseline, mode=baseline_mode, axes=axis[2],
                  title='Beta', show=False, contours=1)
mne.viz.tight_layout()
plt.show()

plt.savefig(res_dir + 'power_topomap_t=0-0p8_' + 'sub-' + sSub + '.pdf')

In [ ]:
# power.plot_joint(baseline=baseline, mode='mean', tmin=None, tmax=None,
#                 timefreqs=[(0., 2.), (0.8, 11.)])

In [ ]:
itc.plot_topo(title='Inter-Trial coherence', vmin=0., vmax=1., cmap='Reds');

In [ ]:
itc.plot([267], baseline=baseline, mode=baseline_mode);
plt.savefig(res_dir + 'itc_MEG2343_' + 'sub-' + sSub + '.pdf')

In [ ]:
itc.plot_topomap(ch_type='mag', tmin=0.1, tmax=0.3, fmin=3.5, fmax=7.,
                baseline=baseline, mode='mean')
# mne.viz.tight_layout()
# plt.savefig(res_dir + 'itc_topomap_' + 'sub-' + sSub + '.pdf')